In [1]:
import requests
from bs4 import BeautifulSoup
검색어='구글증기'
raw = requests.get("https://search.naver.com/search.naver?where=news&query="+검색어)
html = BeautifulSoup(raw.text, "html.parser")

- 하지만 약간의 제안 사항이 있다.

1) API 호출이 일 25,000회로 제한되어 있다. 최대 한 번 호출에 100개의 검색을 가져 올 수 있다.

2) 네이버에서 데이터를 검색하면 뉴스에서만 약 43만건의 데이터가 검색된다. 하지만 최대 1000개만 API를 이용하여 가지고 올 수 있다.

​

In [70]:
import requests
import json
import time   
import pandas as pd

from dotenv import load_dotenv
load_dotenv('/Users/reejungkim/Documents/Git/OpenAPI_Ministry_of_land/.env')

url = 'https://openapi.naver.com/v1/search/news?'
clientId = os.environ.get( 'Naver_News_clientId')
clientSecret = os.environ.get('Naver_News_clientSecret')


### 네이버 뉴스 크롤러 

In [116]:
import os
import sys
import urllib.request
import datetime
import time
import json
#from config import *

#[CODE 1]
def get_request_url(url):
    
    req = urllib.request.Request(url)
    req.add_header("X-Naver-Client-Id", clientId)
    req.add_header("X-Naver-Client-Secret", clientSecret)
    try: 
        response = urllib.request.urlopen(req)
        if response.getcode() == 200:
            print ("[%s] Url Request Success" % datetime.datetime.now())
            return response.read().decode('utf-8')
    except Exception as e:
        print(e)
        print("[%s] Error for URL : %s" % (datetime.datetime.now(), url))
        return None

#[CODE 2]
def getNaverSearchResult(sNode, search_text, page_start, display):
    
    base = "https://openapi.naver.com/v1/search"
    node = "/%s.json" % sNode
    parameters = "?query=%s&start=%s&display=%s" % (urllib.parse.quote(search_text), page_start, display)
    url = base + node + parameters
    
    retData = get_request_url(url)
    
    if (retData == None):
        return None
    else:
        return json.loads(retData)

#[CODE 3]
def getPostData(post, jsonResult):
    
    title = post['title']
    description = post['description']
    org_link = post['originallink']
    link = post['link']

    #Tue, 14 Feb 2017 18:46:00 +0900
    pDate = datetime.datetime.strptime(post['pubDate'],  '%a, %d %b %Y %H:%M:%S +0900')
    pDate = pDate.strftime('%Y-%m-%d %H:%M:%S')
    
    jsonResult.append({'title':title, 'description': description,
                    'org_link':org_link, 'link': org_link, 
                    'pDate':pDate})
    return    

def main():

    jsonResult = []

    # 'news', 'blog', 'cafearticle'
    sNode = 'news'
    search_text = '구글'
    display_count = 100
    
    jsonSearch = getNaverSearchResult(sNode, search_text, 1, display_count)
    
    while ((jsonSearch != None) and (jsonSearch['display'] != 0)):
        for post in jsonSearch['items']:
            getPostData(post, jsonResult)
        
        nStart = jsonSearch['start'] + jsonSearch['display']
        jsonSearch = getNaverSearchResult(sNode, search_text, nStart, display_count)
    
    
    with open('%s_naver_%s.json' % (search_text, sNode), 'w', encoding='utf8') as outfile:
        retJson = json.dumps(jsonResult,
                        indent=4, sort_keys=True,
                        ensure_ascii=False)
        outfile.write(retJson)
        
    print ('%s_naver_%s.json SAVED' % (search_text, sNode))
    
if __name__ == '__main__':
    main()

[2023-01-14 00:52:21.106493] Url Request Success
[2023-01-14 00:52:21.307530] Url Request Success
[2023-01-14 00:52:21.511205] Url Request Success
[2023-01-14 00:52:21.703768] Url Request Success
[2023-01-14 00:52:21.926196] Url Request Success
[2023-01-14 00:52:22.130035] Url Request Success
[2023-01-14 00:52:22.333159] Url Request Success
[2023-01-14 00:52:22.571172] Url Request Success
[2023-01-14 00:52:22.799149] Url Request Success
[2023-01-14 00:52:23.021353] Url Request Success
HTTP Error 400: Bad Request
[2023-01-14 00:52:23.091246] Error for URL : https://openapi.naver.com/v1/search/news.json?query=%EA%B5%AC%EA%B8%80&start=1001&display=100
구글_naver_news.json SAVED


In [89]:
df = pd.read_json(r'/Users/reejungkim/Documents/Git/WebCrawler/구글_naver_news.json')
df.head()


,description,link,org_link,pDate,title
0,필자가 일하는 <b>구글</b>의 사명은 ‘세상의 모든 정보를 잘 정리해서 모든 사...,https://www.hankyung.com/opinion/article/20230...,https://www.hankyung.com/opinion/article/20230...,2023-01-14 00:25:00,[한경에세이] 나는 왜 일하는가?
1,서대문구 세검정로 134 유원하나아파트 앞 상수도관 파열(구경300㎜) (출처=<b...,https://www.gukjenews.com/news/articleView.htm...,https://www.gukjenews.com/news/articleView.htm...,2023-01-14 00:06:00,서울 홍제동 &quot;수도물 단수시간&quot; 상수도관 누수파열
2,14일 영국 가격 비교 웹사이트 컴페어 더 마켓(Compare the market)...,https://www.theguru.co.kr/news/article.html?no...,https://www.theguru.co.kr/news/article.html?no...,2023-01-14 00:02:00,전 세계에서 가장 많이 검색된 자동차 브랜드?
3,이와 관련해 유튜브를 운영하는 <b>구글</b>의 언론 담당자는 해당 계정이 <b>...,https://www.chosun.com/politics/north_korea/20...,https://www.chosun.com/politics/north_korea/20...,2023-01-13 23:51:00,평양 ‘브이로그’ 유튜버 또 등장... 유창한 영어 쓰는 ‘유미’의 정체는
4,"게임은 물론, 소셜미디어와 비디오 스트리밍 앱은 해당 기간 <b>구글</b>플레이와...",https://www.donga.com/news/article/all/2023011...,https://www.donga.com/news/article/all/2023011...,2023-01-13 22:35:00,[기고] 2023년 &apos;다섯번째&apos; 분기의 마케팅 극대화 방법은?
